In [1]:
import numpy as np
import pandas as pd
import trimesh as tm
from matplotlib import pyplot as plt

In [2]:
def load_template(file_path):
        faces = []
        normals = []
        i = 0
        with open(file_path) as file:
            for line in file:
                if line[0] == "f":
                    line_split = line[1:].split()
                    line_split = np.array([i.split("//") for i in line_split])
                    face_temp = list(map(int, line_split[:, 0]))
                    #normal_temp = list(map(int, line_split[:, 1]))
                    faces.append(face_temp)
                    #normals.append(normal_temp)
                    i += 1
        return np.array(faces), np.array(normals)

def load_mesh(file_path):
    vertex = []
    with open(file_path, "r") as f:
        for line in f:
            if line[0] == '#':
                continue
            elif "v " in line:
                line.replace('\n', ' ')
                tmp = list(map(float, line[1:].split()))
                vertex.append(tmp)
            else:
                break
    vertex = np.array(vertex)
    return vertex

In [3]:
vertices = load_mesh('../curves-modules/mean_female_pose_1.obj')
faces, _ = load_template('../curves-modules/mean_female_pose_1.obj')
faces-=1

In [4]:
levels = np.linspace(-1.5, 1.5,10000)

In [5]:
mesh = tm.Trimesh(vertices=vertices, faces=faces, process=False)

In [6]:
sections = mesh.section_multiplane(plane_origin=mesh.centroid, plane_normal=[0,1,0], heights=levels)

In [53]:
import cv2


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [73]:
fourcc = cv2.VideoWriter_fourcc(*'MJPG')    
video = cv2.VideoWriter("test.avi",fourcc, 10,(800,600))
for section in sections:
    if section:
        image = section.rasterize(1,0, resolution=(800,600))
        image = np.array(image, dtype=np.uint8)
        video.write(image)
video.release()
cv2.destroyAllWindows()

In [ ]:
np.array(values).max()

18.915353544897286